## 1. 데이터 로드

In [3]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc3_0_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_csv/"+'gcc3'+"_0_32.csv", index_col=0)

# 형태 출력
print(gcc3_0_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc3_0_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc3_0_32.head()

(2129359, 2)
reset_index 완료
input data shape


,bin,label
0,55,1
1,89,0
2,e5,0
3,83,0
4,ec,0


## 2. Data 확인

In [4]:
# DATA 모양 확인
gcc3_0_32.values
print(gcc3_0_32.shape)

# 함수의 갯수
gcc3_0_32['label'].value_counts()

# 함수 평균길이
gcc3_0_32['label'].value_counts()[0]/gcc3_0_32['label'].value_counts()[1]

array([['55', 1],
       ['89', 0],
       ['e5', 0],
       ...,
       ['fc', 0],
       ['c9', 0],
       ['c3', 0]], dtype=object)

(2129359, 2)


0    2125055
1       4304
Name: label, dtype: int64

493.7395446096654

## 3. 함수의 갯수, 바이트 갯수의 빈공간 np.array 생성 

In [5]:
x = np.full((gcc3_0_32['label'].value_counts()[1],30000), None, dtype='object')
x.shape
x.dtype

(4304, 30000)

dtype('O')

## 4. 함수 빈공간에 함수별로 한줄씩 값 대입

In [6]:
data = gcc3_0_32.values

row = -1
col = 0
col_list=[]

# data => [55,1], [89,1], [e5,1] ....
for i in data:
    # i[0] = 55, 89, e5... /  i[1] = 1, 0, 0, ....
    if i[1]==1:
        col_list.append(col) # col_list의 각 함수 길이들 넣는다.
        
        row+=1
        col=0
        x[row][col] = i[0]
#        print(row,col, end='/') # 오류났을 때 풀어보면 몇번째 행에서 값을 초과하는지 알 수 있음.
    else:
        x[row][col+1] = i[0]
        #if row >=1385:
        #    print(row,col, end=' ')
        col+=1
    
print('대입 완료')    

대입 완료


## 5. 최적의 x 찾기

In [14]:
# 예시
x[:, :6]
#print(len(col_list),'\n')

# 최소 3gram ~ 7gram 까지확인
for j in range(3, 7):
    tuple_ls = []
    for k in x[:, :j]:
        tuple_ls.append(tuple(k))

    set_ls = set(tuple_ls)
    print(j, 'gram일 때 ', len(set_ls), '쌍')

array([['55', '89', 'e5', '83', 'ec', '28'],
       ['55', '89', 'e5', '83', 'ec', '18'],
       ['55', '89', 'e5', '83', 'ec', '18'],
       ...,
       ['55', '89', 'e5', '83', 'ec', '18'],
       ['55', '89', 'e5', '83', 'ec', '18'],
       ['55', '89', 'e5', '83', 'ec', '18']], dtype=object)

3 gram일 때  1 쌍
4 gram일 때  13 쌍
5 gram일 때  36 쌍
6 gram일 때  99 쌍


In [8]:
# 임의 테스트
tls = []
for t in x[:, :4]:
    tls.append(tuple(t))
    
stls = set(tls)
print(len(stls),'개')
sorted(stls)

13 개


[('55', '89', 'e5', '0f'),
 ('55', '89', 'e5', '53'),
 ('55', '89', 'e5', '56'),
 ('55', '89', 'e5', '57'),
 ('55', '89', 'e5', '5d'),
 ('55', '89', 'e5', '81'),
 ('55', '89', 'e5', '83'),
 ('55', '89', 'e5', '8b'),
 ('55', '89', 'e5', 'a1'),
 ('55', '89', 'e5', 'b8'),
 ('55', '89', 'e5', 'c6'),
 ('55', '89', 'e5', 'c7'),
 ('55', '89', 'e5', 'e8')]